# Team Analysis
==========================================================================================

### Thinking
* Players on better teams will play with better players and therefore will perform better
    * Opponents will not be able to focus in on one specific player
* Playing on a team with good defense will only for more offensive snaps
* A good team plays to its strengths: good players will get the ball more
    * Having a smart offensive coordinator or head coach that has a playbook that involves their best players is ideal
* However, a good team may simply just have good players
    * This means that sometimes, your player will perform well regardless of team: they are simply effective 
    * In addition, there is only one ball: stats cannot be shared
        * Playing on a poor team would allow one great player to take in all the stats
    * Does playing on a good team improve or decrease your fantasy production?
    * In reality, this would mainly apply to teams with multiple superstars
   
==========================================================================================
### Data Collection
* Data is available from 2016-2021 (2022, but I don't want to include that data)
* Players have their stats available in this same time period, but advanced stats and defensive aren't tracked until 2018
    * Therefore, I will exclude looking at advanced stats in general in the first bit
    
==========================================================================================
### Method of Analysis
##### Numbers:
* Will calculate the difference of a player's draft position and final standing and its 
    * For example: a player with an ADP (Average Draft Position) of RB12 but finishes the year as the RB5 would be +7
        * Flipping the ADP and Finish would be a -7 rating
        * However, finishing at RB5 from ADP12 is more important than finishing RB24 from ADP31
            * The gap in points is simply too large
            
##### Cleaning + Preparing
* Bring in the data and store in pandas dataframes
    * Need to bring in:
        * ADP (2016-2021Just Overall)
        * Final Seasonal Numbers
        * RedZone Passing, Receiving, Rushing
        * TeamOffense:
            * Conversions
            * Drive averages
            * Passing Offense
            * Rushing Offense
            * Scoring Offense
* Create a Hashing System to link up team names with their abreviations
* Link up player data (Player Team -> Team scoring/passing/receiving/rushing)
* Divide up players by position for comparison
* Scale the numbers (for SVM)

##### Clustering
* Graph a couple of seemingly import attributes
* See if KMeans or MeanShift can cluster the data into groups, may take a long time

##### Modeling
* Will use a SVM and DecisionTree (Maybe RandomForest) to see if there is any trends
* Test on 2022 Seasonal Data while training on these 5-6 years

In [56]:
### Selecting data to load into list of pandas dataframes
import numpy as np
import pandas as pd
adp = [] #ADP
rnk = [] #End of Season FantasyRankings
rzp = [] #RedZone Passing
rzru = [] #RedZone Rushing
rzre = [] #RedZone Receiving
convs = [] #Conversions
das = [] #driveAverages
pO = [] #passingOffense
rO = [] #rushingOffense
sO = [] #scoringOffense
tO = [] #teamOffense
for i in range(2016, 2022):
    address = "Data/ADP/FantasyPros_" + str(i) + "_Overall_ADP_Rankings.csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    adp.append(temp)
    address = "Data/FantasyRankings/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    rnk.append(temp)
    address = "Data/PlayerStats/Passing/redZonePassing/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzp.append(temp)
    address = "Data/PlayerStats/Rushing/redZoneRushing/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzru.append(temp)
    address = "Data/PlayerStats/Receiving/redZoneReceiving/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzre.append(temp)
    address = "Data/TeamOffense/conversions/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    convs.append(temp)
    address = "Data/TeamOffense/driveAverages/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    das.append(temp)
    address = "Data/TeamOffense/passingOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    pO.append(temp)
    address = "Data/TeamOffense/rushingOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    rO.append(temp)
    address = "Data/TeamOffense/scoringOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    sO.append(temp)
    address = "Data/TeamOffense/teamOffense/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[1])
    temp.fillna(0, inplace=True)
    tO.append(temp)
teamdata = []
for i in range(0, 6):
    df = pd.merge(convs[i], das[i], left_index=True, right_index=True)
    df = pd.merge(df, pO[i], left_index=True, right_index=True)
    df = pd.merge(df, rO[i], left_index=True, right_index=True)
    df = pd.merge(df, sO[i], left_index=True, right_index=True)
    df = pd.merge(df, tO[i], left_index=True, right_index=True)
    teamdata.append(df)
for i in range(5):
    address = "Data/Merged/" + str(i+ 2017) + ".csv"
    teamdata[i].to_csv(address)
for i in range(6):
    adp[i] = adp[i][['Player', 'POS', 'AVG']]
    rnk[i].set_index('Player')
    rnk[i] = rnk[i].drop(['Unnamed: 0'], axis=1, inplace=True)
for i in range(6):
    rnk[i]['Player'] = rnk[i]['Player'].str.replace('*', '')
    rnk[i]['Player'] = rnk[i]['Player'].str.replace('+', '')
for i in range(6):
    t1 = adp[i].copy(deep=True)
    t2 = rnk[i].copy(deep=True)
    t3 = rzp[i].copy(deep=True)
    t4 = rzru[i].copy(deep=True)
    t5 = rzre[i].copy(deep=True)

    t1 = t1.join(t2, on='Player')
    t1 = t1.join(t3, on='Player', lsuffix='rank', rsuffix='rzpa')
    t1 = t1.join(t4, on='Player', lsuffix='rzpa', rsuffix='rzru')
    t1 = t1.join(t5, on='Player', lsuffix='rzru', rsuffix='rzre')
    t1.fillna(0, inplace=True)
    address = "Data/MergedPlayers/" + str(i+2016) + ".csv"
    t1.to_csv(address)

In [29]:
# Creating a set of the teams
# Due to moving of team and team name changes, this is not as simple as it would seem
namesHash = {"DET": "Detroit Lions",
"GNB": "Green Bay Packers",
"MIN": "Minnesota Vikings",
"JAX": "Jacksonville Jaguars",
"LVR": "Las Vegas Raiders",
"PHI": "Philadelphia Eagles",
"NOR": "New Orleans Saints",
"SFO": "San Francisco 49ers",
"HOU": "Houston Texans",
"MIA": "Miami Dolphins",
"PIT": "Pittsburgh Steelers",
"SEA": "Seattle Seahawks",
"CHI": "Chicago Bears",
"IND": "Indianapolis Colts",
"TEN": "Tennessee Titans",
"NYJ": "New York Jets",
"LAR": "Los Angeles Rams",
"DAL": "Dallas Cowboys",
"2TM": "",
"KAN": "Kansas City Chiefs",
"SDG": "San Diego Chargers",
"3TM": "",
"LAC": "Los Angeles Chargers",
"BUF": "Buffalo Bills",
"BAL": "Baltimore Ravens",
"ATL": "Atlanta Falcons",
"OAK": "Oakland Raiders",
"DEN": "Denver Broncos",
"CLE": "Cleveland Browns",
"NWE": "New England Patriots",
"TAM": "Tampa Bay Buccaneers",
"NYG": "New York Giants",
"CIN": "Cincinnati Bengals",
"ARI": "Arizona Cardinals",
"WAS": "Washington Redskins Washington Football Team",
"CAR": "Carolina Panthers",}      

adp = [] #ADP
rnk = [] #End of Season FantasyRankings
rzp = [] #RedZone Passing
rzru = [] #RedZone Rushing
rzre = [] #RedZone Receiving

In [23]:
import numpy as np
import pandas as pd

rnk = []
team = []

for i in range(2016, 2022):
    address = "Data/Merged/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[2])
    temp.fillna(0, inplace=True)
    rnk.append(temp)
    address = "Data/MergedPlayers/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[3])
    temp.fillna(0, inplace=True)
    team.append(temp)

In [48]:
import numpy as np
import pandas as pd
adp = [] #ADP
rnk = [] #End of Season FantasyRankings
rzp = [] #RedZone Passing
rzru = [] #RedZone Rushing
rzre = [] #RedZone Receiving
for i in range(2016, 2022):
    address = "Data/ADP/FantasyPros_" + str(i) + "_Overall_ADP_Rankings.csv"
    temp = pd.read_csv(address, index_col=[2])
    temp.drop(['Unnamed: 0'], axis=1, inplace=True)
    temp.fillna(0, inplace=True)
    adp.append(temp)
    address = "Data/FantasyRankings/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[3])
    temp.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
    temp.fillna(0, inplace=True)
    rnk.append(temp)
    address = "Data/PlayerStats/Passing/redZonePassing/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzp.append(temp)
    address = "Data/PlayerStats/Rushing/redZoneRushing/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzru.append(temp)
    address = "Data/PlayerStats/Receiving/redZoneReceiving/" + str(i) + ".csv"
    temp = pd.read_csv(address, index_col=[0])
    temp.fillna(0, inplace=True)
    rzre.append(temp)



In [50]:
sums = []
for i in range(6):
    t1 = adp[i].copy(deep=True)
    t2 = rnk[i].copy(deep=True)
    t3 = rzp[i].copy(deep=True)
    t4 = rzru[i].copy(deep=True)
    t5 = rzre[i].copy(deep=True)

    t1 = t1.join(t2, on='Player')
    t1 = t1.join(t3, on='Player', lsuffix='rank', rsuffix='rzpa')
    t1 = t1.join(t4, on='Player', lsuffix='rzpa', rsuffix='rzru')
    t1 = t1.join(t5, on='Player', lsuffix='rzru', rsuffix='rzre')
    t1.fillna(0, inplace=True)
    address = "Data/MergedPlayers/" + str(i+2016) + ".csv"
    t1.to_csv(address)
    sums.append(t1)

In [52]:
for i in range(6):
    sums[0]['Difference'] = sums[0]['AVG'] - sums['']

In [53]:
sums[0]['Difference'].head()

Player
Antonio Brown        11.0
Odell Beckham Jr.    12.3
Todd Gurley          46.5
Julio Jones          16.7
David Johnson        -3.5
Name: Difference, dtype: float64

In [54]:
sums[0].head()

,Rank,POS,AVG,Rk,Tmrank,FantPos,Age,G,GS,Cmprank,...,%Tgt,Tgt.1,Rec.1,Ctch%.1,Yds.1rzre,TD.1rzre,%Tgt.1,Link,-9999rzre,Difference
Player,,,,,,,,,,,,,,,,,,,,,
Antonio Brown,1.0,WR1,1.0,12.0,PIT,WR,28.0,15.0,15.0,0.0,...,23.4%,7.0,3.0,42.86%,13.0,3.0,24.1%,All Antonio Brown red zone receiving plays,BrowAn04,11.0
Odell Beckham Jr.,2.0,WR2,2.7,15.0,NYG,WR,24.0,16.0,16.0,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0,0,12.3
Todd Gurley,3.0,RB1,3.5,50.0,LAR,RB,22.0,16.0,16.0,0.0,...,17.6%,0.0,0.0,0,0.0,0.0,0.0%,All Todd Gurley red zone receiving plays,GurlTo01,46.5
Julio Jones,4.0,WR3,4.3,21.0,ATL,WR,27.0,14.0,14.0,0.0,...,9.7%,6.0,3.0,50.00%,11.0,2.0,12.0%,All Julio Jones red zone receiving plays,JoneJu02,16.7
David Johnson,5.0,RB2,4.5,1.0,ARI,RB,25.0,16.0,16.0,0.0,...,16.7%,5.0,3.0,60.00%,10.0,3.0,12.8%,All David Johnson red zone receiving plays,JohnDa08,-3.5
